This query is based on patients seen at the Hugh Kaul Precision Medicine Institute.  

Two PMI patients with developmental and speech delays, as well as seizures have been identified with missense variants in the [RHOBTB2](https://docs.google.com/presentation/d/1393qJ_Pcsl-hkwrsScqNLycEOuk-7dDkp5ey4UuEBbA/edit#slide=id.gd71774717d_0_6) gene.   [RHOBTB2 Syndrome](https://rarechromo.org/media/singlegeneinfo/Single%20Gene%20Disorder%20Guides/RHOBTB2%20syndrome%20QFN.pdf) has only been known since 2018, and as of 2020, only 13 children with such variations have been identified.

The syndrome is thought to be related to gain of function missense mutations in RHOBTB2, so the PMI researchers want to use Translator to find interventions that will reduce the expression or activity of RHOBTB2 with the aim of ameliorating symptoms.

The following functions are used send queries to Translator and display the results

In [1]:
#imports
import json
import requests
from collections import defaultdict
import pandas as pd
import copy
from datetime import datetime as dt

In [2]:
#ARS functions
def submit_to_ars(m,ars_url='https://ars.ci.transltr.io/ars/api',arax_url='https://arax.ncats.io'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

def retrieve_ars_results(mid,ars_url='https://ars.ci.transltr.io/ars/api'):
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    for child in j['children']:
        print(child['status'])
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                print(e)
                child['status'] = 'ARS Error'
        else:
            nresults = 0
        print( child['status'], child['actor']['agent'],nresults )
    return results

In [3]:
#utils
def printjson(j):
    print(json.dumps(j,indent=4))
def print_json(j):
    printjson(j)

The story continues...

We first want to know if there are any chemicals that can reduce RHOBTB2 activity or amount.   This is a simple query that looks for chemicals with such an effect:

In [4]:
with open('A.0_RHOBTB2_direct.json','r') as inf:
    query_0 = json.load(inf)
printjson(query_0)

{
    "message": {
        "query_graph": {
            "edges": {
                "e01": {
                    "object": "n0",
                    "subject": "n1",
                    "predicates": [
                        "biolink:entity_negatively_regulates_entity"
                    ]
                }
            },
            "nodes": {
                "n0": {
                    "ids": [
                        "NCBIGene:23221"
                    ],
                    "categories": [
                        "biolink:Gene"
                    ]
                },
                "n1": {
                    "categories": [
                        "biolink:SmallMolecule"
                    ]
                }
            }
        }
    }
}


The query above links a Gene, specified with an NCBIGene identifier, with an unnamed SmallMolecule.   Furthermore, it asks that relationship(predicate) between these two things is that the SmallMolecule "decreases" the Gene.  Here "decreases" is defined as one of any number of relationships in our shared data model, the [biolink model](https://github.com/biolink/biolink-model/blob/master/biolink-model.yaml).  We can send this query to Translator and view the results.

In [5]:
pk_0 = submit_to_ars(query_0)

https://arax.ncats.io/?source=ARS&id=2c3211dd-b27c-488c-97ef-93e57d6d47ef


Unfortunately, we find that there are no chemicals that inhibit, downregulate, lower the amount of, or decrease the expression of RHOBTB2 known to Translator.  This may not be uncommon for a poorly studied gene.

To affect RHOBTB2, then, we will need to take a more indirect approach.  Now, we will attempt to find genes that can downregulate RHOBTB2, and then find SmallMolecule that increase the activity of those genes.

In [17]:
with open('A.Multiomics_BigGIM_DR_KP_RHOBTB2.json','r') as inf:
    query_1 = json.load(inf)
printjson(query_1)

{
    "message": {
        "query_graph": {
            "nodes": {
                "n0": {
                    "ids": [
                        "NCBIGene:23221"
                    ],
                    "categories": [
                        "biolink:Gene"
                    ]
                },
                "n1": {
                    "categories": [
                        "biolink:SmallMolecule"
                    ]
                }
            },
            "edges": {
                "e01": {
                    "subject": "n0",
                    "object": "n1",
                    "predicates": [
                        "biolink:affects_response_to"
                    ]
                }
            }
        }
    }
}


In [18]:
pk_1 = submit_to_ars(query_1)

https://arax.ncats.io/?source=ARS&id=db4bfd09-084f-449e-8495-d524092ebef6


The difference between query_0_1 and query_0 is the reverse direction between the Gene and SmallMolecule. We found only a few results such as  <font color='blue'>RUXOLITINIB</font>. Evidence from the biothings-multiomics-biggim-drugresponse KP shows cell lines with RHOBTB2 mutations shows significant lower IC50 values compared to cell lines with wildtype RHOBTB2. RUXOLITINIB is JAK inhibitors. JAK-STAT3 signaling has been reported to cross-talk with Rho GTPase signaling in the context of cancer [PMC4125382]. The underline mechanism of the higher sensitivity of JAK inhibitor RUXOLITINIB in the RHOBTB2 mutated samples is still not clear. More knowledge graphs or multiple hop queries will be needed. </br> Edit by Guangrong Qin 

In [13]:
with open('A.2_RHOBTB2_twohop.json','r') as inf:
    query_2 = json.load(inf)
printjson(query_2)

{
    "message": {
        "query_graph": {
            "edges": {
                "e01": {
                    "object": "n0",
                    "subject": "n1",
                    "predicates": [
                        "biolink:entity_regulates_entity",
                        "biolink:genetically_interacts_with"
                    ]
                },
                "e02": {
                    "object": "n1",
                    "subject": "n2",
                    "predicates": [
                        "biolink:related_to"
                    ]
                }
            },
            "nodes": {
                "n0": {
                    "ids": [
                        "NCBIGene:23221"
                    ],
                    "categories": [
                        "biolink:Gene"
                    ]
                },
                "n1": {
                    "categories": [
                        "biolink:Gene"
                    ]
                },
        

In [14]:
pk_2 = submit_to_ars(query_2)

https://arax.ncats.io/?source=ARS&id=3f68f0a5-8fc7-43bc-b458-0ccf23938d00
